In [1]:
#@title Convert XML file to CSV file (SKIP THIS CELL)

import xml.etree.ElementTree as ET
import csv

tree = ET.parse('OpenStreetMap Trace for a Sparse Traffic.xml')  # replace with your file path
root = tree.getroot()

# open a file for writing
with open('sparse.csv', 'w', newline='') as out_csv:
    writer = csv.writer(out_csv)
    writer.writerow(["time", "id", "x", "y", "angle", "type", "speed", "pos", "lane", "slope"])

    for timestep in root.findall('timestep'):
        time = timestep.get('time')
        for vehicle in timestep.findall('vehicle'):
            id = vehicle.get('id')
            x = vehicle.get('x')
            y = vehicle.get('y')
            angle = vehicle.get('angle')
            type = vehicle.get('type')
            speed = vehicle.get('speed')
            pos = vehicle.get('pos')
            lane = vehicle.get('lane')
            slope = vehicle.get('slope')
            writer.writerow([time, id, x, y, angle, type, speed, pos, lane, slope])

In [3]:
#@title Read the input data (Decompress sparse.zip if you haven't already) - (SKIP THIS CELL IF YOU ARE WORKING WITH preprocessed_sparse.csv)

import pandas as pd

# from google.colab import drive
# drive.mount('/content/drive')

# Load data
data = pd.read_csv('sparse.csv')
# data = pd.read_csv('/content/drive/MyDrive/2024-05- 7088CEM (ANN Module)/Assignments/handover_prediction/sparse.csv')
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1483385 entries, 0 to 1483384
Data columns (total 10 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   time    1483385 non-null  float64
 1   id      1483385 non-null  object 
 2   x       1483385 non-null  float64
 3   y       1483385 non-null  float64
 4   angle   1483385 non-null  float64
 5   type    1483385 non-null  object 
 6   speed   1483385 non-null  float64
 7   pos     1483385 non-null  float64
 8   lane    1483385 non-null  object 
 9   slope   1483385 non-null  float64
dtypes: float64(7), object(3)
memory usage: 113.2+ MB


,time,x,y,angle,speed,pos,slope
count,1.483385e+06,1.483385e+06,1.483385e+06,1.483385e+06,1.483385e+06,1.483385e+06,1483385.0
mean,3.154547e+03,5.425987e+02,1.616521e+03,1.624481e+02,2.809118e+00,5.510633e+01,0.0
std,1.609442e+03,1.629608e+02,2.089379e+02,1.073253e+02,5.907708e+00,8.076256e+01,0.0
min,0.000000e+00,2.140000e+00,8.206800e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,2.000000e+03,4.611200e+02,1.559000e+03,9.028000e+01,0.000000e+00,6.800000e+00,0.0
50%,3.121000e+03,4.780700e+02,1.683820e+03,1.780200e+02,0.000000e+00,2.232000e+01,0.0
75%,4.001000e+03,5.699000e+02,1.697190e+03,2.709600e+02,4.000000e-01,5.922000e+01,0.0
max,8.219000e+03,1.420580e+03,2.212710e+03,3.600000e+02,3.738000e+01,4.171000e+02,0.0


In [4]:
#@title Preprocess data (getting rid of unnecessary columns) (SKIP THIS CELL IF YOU ARE WORKING WITH preprocessed_sparse.csv (You can decompress the zip file with the same name))
# Convert the 'time' column to float and then to integer
data['time'] = data['time'].astype(float).astype(int)

# Remove 'veh' from 'id' column and convert to integer
data['id'] = data['id'].str.replace('veh', '').astype(int)

# Remove specified columns
columns_to_remove = ['type', 'lane', 'slope', 'pos']
data = data.drop(columns=columns_to_remove)

# Display the first few rows of the processed data
data.head(20)
data.info()
data.describe()

# Save the preprocessed DataFrame to a CSV file for future use
# output_path = '/content/drive/MyDrive/2024-05-7088CEM (ANN Module)/Assignments/handover_prediction/preprocessed_sparse.csv'
output_path = 'preprocessed_sparse.csv'
data.to_csv(output_path, index=False)

print(f"Preprocessed data saved to {output_path}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1483385 entries, 0 to 1483384
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   time    1483385 non-null  int64  
 1   id      1483385 non-null  int64  
 2   x       1483385 non-null  float64
 3   y       1483385 non-null  float64
 4   angle   1483385 non-null  float64
 5   speed   1483385 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 67.9 MB
Preprocessed data saved to preprocessed_sparse.csv


In [5]:
#@title Preprocesing: Preparing train and test datasets (Save sequenced data) (YOU CAN SKIP THIS BY DECOMPRESSING sequences_vehicle_ids.zip)
import pickle

sequence_length = 5

def create_sequences(data, sequence_length):
    sequences = [] # each seqence has the properties of one vehicle in different time steps (sequential)
    vehicle_ids = []
    for veh_id, group in data.groupby('id'):
        group = group.sort_values(by='time')
        for i in range(len(group) - sequence_length):
            seq = group.iloc[i:i+sequence_length]
            sequences.append((seq[['x', 'y', 'speed', 'angle']].values, seq[['x', 'y']].values[-1]))
            vehicle_ids.append(veh_id)
    return sequences, vehicle_ids

sequences, vehicle_ids = create_sequences(data, sequence_length)
# Save sequences and vehicle IDs to a file

# with open('/content/drive/MyDrive/2024-05- 7088CEM (ANN Module)/Assignments/handover_prediction/sequences_vehicle_ids.pkl', 'wb') as f:
with open('sequences_vehicle_ids.pkl', 'wb') as f:
    pickle.dump((sequences, vehicle_ids), f)

# how to load it later?
# with open('sequences_vehicle_ids.pkl', 'rb') as f:
#     sequences, vehicle_ids = pickle.load(f)